In [1]:
%load_ext autoreload
%autoreload 2

# Chat with website
Generates a pipeline to read a web page and ask questions about its content.

In [2]:
from haystack import Pipeline, PredefinedPipeline

pipeline = Pipeline.from_template(PredefinedPipeline.CHAT_WITH_WEBSITE)
pipeline.run({"fetcher": {"urls": ["https://haystack.deepset.ai:"]}, "prompt": {"query": "what is Haystack?"}})


PipelineUnmarshalError: Error unmarshalling pipeline: Couldn't deserialize component 'llm' of class 'OpenAIGenerator' with the following data: {'init_parameters': {'api_base_url': None, 'api_key': EnvVarSecret(_env_vars=('OPENAI_API_KEY',), _strict=True, _type=<SecretType.ENV_VAR: 'env_var'>), 'generation_kwargs': {}, 'model': 'gpt-4o-mini', 'streaming_callback': None, 'system_prompt': None}, 'type': 'haystack.components.generators.openai.OpenAIGenerator'}. Possible reasons include malformed serialized data, mismatch between the serialized component and the loaded one (due to a breaking change, see https://github.com/deepset-ai/haystack/releases), etc.
Source:
components:
  converter:
    type: haystack.components.converters.html.HTMLToDocument
    init_parameters:
      extraction_kwargs: null

  fetcher:
    init_parameters:
      raise_on_failure: true
      retry_attempts: 2
      timeout: 3
      user_agents:
      - haystack/LinkContentFetcher/2.0.0b8
    type: haystack.component

# Generative QA
Generates a simple pipeline to ask a generic query using an OpenAIGenerator.

In [3]:
from haystack import Pipeline, PredefinedPipeline

pipeline = Pipeline.from_template(PredefinedPipeline.GENERATIVE_QA)
pipeline.run({"prompt_builder":{"question":"Where is Rome?"}})

PipelineUnmarshalError: Error unmarshalling pipeline: Couldn't deserialize component 'generator' of class 'OpenAIGenerator' with the following data: {'init_parameters': {'api_key': EnvVarSecret(_env_vars=('OPENAI_API_KEY',), _strict=True, _type=<SecretType.ENV_VAR: 'env_var'>), 'model': 'gpt-4o-mini'}, 'type': 'haystack.components.generators.openai.OpenAIGenerator'}. Possible reasons include malformed serialized data, mismatch between the serialized component and the loaded one (due to a breaking change, see https://github.com/deepset-ai/haystack/releases), etc.
Source:
---

components:
  generator:
    init_parameters:
      api_key:
        env_vars: [ "OPENAI_API_KEY" ]
        strict: true
        type: "env_var"
      model: "gpt-4o-mini"
    type: "haystack.components.generators.openai.OpenAIGenerator"

  prompt_builder:
    init_parameters:
      template: "Answer the question {{question}}.\n Answer:\n"
    type: "haystack.components.builders.prompt_builder.PromptBuilder"


conn

# Indexing
Generates a pipeline that imports documents from one or more text files, creates the embeddings for each of them, and finally stores them in an `InMemoryDocumentStore`.

In [4]:
from haystack import Pipeline, PredefinedPipeline

pipeline = Pipeline.from_template(PredefinedPipeline.INDEXING)
result = pipe.run({"converter": {"sources": ["some_file.txt"]}})

PipelineUnmarshalError: Error unmarshalling pipeline: Couldn't deserialize component 'embedder' of class 'OpenAIDocumentEmbedder' with the following data: {'init_parameters': {'api_base_url': None, 'api_key': EnvVarSecret(_env_vars=('OPENAI_API_KEY',), _strict=True, _type=<SecretType.ENV_VAR: 'env_var'>), 'batch_size': 32, 'dimensions': None, 'embedding_separator': '\\n', 'meta_fields_to_embed': [], 'model': 'text-embedding-ada-002', 'organization': None, 'prefix': '', 'progress_bar': True, 'suffix': ''}, 'type': 'haystack.components.embedders.openai_document_embedder.OpenAIDocumentEmbedder'}. Possible reasons include malformed serialized data, mismatch between the serialized component and the loaded one (due to a breaking change, see https://github.com/deepset-ai/haystack/releases), etc.
Source:
---

components:
  cleaner:
    init_parameters:
      remove_empty_lines: true
      remove_extra_whitespaces: true
      remove_regex: null
      remove_repeated_substrings: false
      remo

# RAG
Generates a RAG pipeline using data that was previously indexed (you can use the Indexing template).

In [5]:
from haystack import Pipeline, PredefinedPipeline

pipeline = Pipeline.from_template(PredefinedPipeline.RAG)
pipeline.run({"text_embedder": {"text": "A question about your documents"}})

PipelineUnmarshalError: Error unmarshalling pipeline: Couldn't deserialize component 'llm' of class 'OpenAIGenerator' with the following data: {'init_parameters': {'api_base_url': None, 'api_key': EnvVarSecret(_env_vars=('OPENAI_API_KEY',), _strict=True, _type=<SecretType.ENV_VAR: 'env_var'>), 'generation_kwargs': {}, 'model': 'gpt-4o-mini', 'streaming_callback': None, 'system_prompt': None}, 'type': 'haystack.components.generators.openai.OpenAIGenerator'}. Possible reasons include malformed serialized data, mismatch between the serialized component and the loaded one (due to a breaking change, see https://github.com/deepset-ai/haystack/releases), etc.
Source:
---

components:
  llm:
    init_parameters:
      api_base_url: null
      api_key:
        env_vars:
        - OPENAI_API_KEY
        strict: true
        type: env_var
      generation_kwargs: {}
      model: gpt-4o-mini
      streaming_callback: null
      system_prompt: null
    type: haystack.components.generators.openai.Op